In [22]:
import nltk
import re
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline

Absa_refined dataset is created by applying the following steps on the  womens-ecommerce-clothing-reviews dataset downloaded from kaggle
* Pre-processing
* Aspect extraction from each reviews
* Apply VADER sentiment analysis to find the sentiment corresponding to the aspects extracted from the reviews.
* Created a new dataset named "absa_refined_latest.csv" which contains the attributes which are essential for further Computations.

In [23]:
df=pd.read_csv(r"F:/AMRITA MTECH/Datascience for NLP/absa_refined_latest.csv")

In [3]:
df.columns

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Rating', 'Review', 'Division Name',
       'aspect_keywords', 'sentiment', 'Department Name', 'Class Name'],
      dtype='object')

In [24]:
df1=df.iloc[:,4:8]

In [25]:
df1

,Review,Division Name,aspect_keywords,sentiment
0,Absolutely wonderful - silky and sexy and com...,Initmates,[],pos
1,Love this dress! it's sooo pretty. i happen...,General,"['never ordered', 'ordered online', 'true midi']",pos
2,Some major design flawsI had such high hopes f...,General,"['major design', 'initially ordered', 'such ho...",pos
3,"My favorite buy!I love, love, love this jumpsu...",General Petite,"['favorite buy!I love', 'great compliments']",pos
4,Flattering shirtThis shirt is very flattering ...,General,"['adjustable front tie', 'Flattering shirt', '...",pos
...,...,...,...,...
22623,Great dress for many occasionsI was very happy...,General Petite,"['great price', 'easy to slip', 'Great dress',...",pos
22624,Wish it was made of cottonIt reminds me of mat...,General Petite,"['long sleeves', 'soft material', 'nice sleeve...",pos
22625,"Cute, but see throughThis fit well, but the to...",General Petite,"['different fabric', 'fit well', 'never worked...",pos
22626,"Very cute dress, perfect for summer parties an...",General,"['much cheaper t', 'cheaper t', 'cute dress', ...",pos


In [26]:
df1=df1.drop(['Division Name','aspect_keywords'],axis=1)

In [27]:
df1

,Review,sentiment
0,Absolutely wonderful - silky and sexy and com...,pos
1,Love this dress! it's sooo pretty. i happen...,pos
2,Some major design flawsI had such high hopes f...,pos
3,"My favorite buy!I love, love, love this jumpsu...",pos
4,Flattering shirtThis shirt is very flattering ...,pos
...,...,...
22623,Great dress for many occasionsI was very happy...,pos
22624,Wish it was made of cottonIt reminds me of mat...,pos
22625,"Cute, but see throughThis fit well, but the to...",pos
22626,"Very cute dress, perfect for summer parties an...",pos


In [8]:
X = df.Review
Y = df.sentiment
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [9]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [10]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [11]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [12]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 50)           50000     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257   

In [13]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 15386 samples, validate on 3847 samples
Epoch 1/10
15386/15386 [==============================] - 68s 4ms/sample - loss: 0.2705 - accuracy: 0.9262 - val_loss: 0.2380 - val_accuracy: 0.9298
Epoch 2/10
15386/15386 [==============================] - 53s 3ms/sample - loss: 0.2288 - accuracy: 0.9291 - val_loss: 0.2251 - val_accuracy: 0.9301
Epoch 3/10
15386/15386 [==============================] - 52s 3ms/sample - loss: 0.2158 - accuracy: 0.9300 - val_loss: 0.2243 - val_accuracy: 0.9301
Epoch 4/10
15386/15386 [==============================] - 51s 3ms/sample - loss: 0.2022 - accuracy: 0.9308 - val_loss: 0.2179 - val_accuracy: 0.9290
Epoch 5/10
15386/15386 [==============================] - 56s 4ms/sample - loss: 0.1917 - accuracy: 0.9305 - val_loss: 0.2189 - val_accuracy: 0.9290


In [14]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [15]:
accr = model.evaluate(test_sequences_matrix,Y_test)

3395/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [16]:
print('Test set\n     Accuracy: {:0.3f}'.format(accr[1]))

Test set
     Accuracy: 0.929


In [17]:
sequences_matrix[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  26,  55,
       373,  36, 102, 119,  95,  56, 120, 122, 102,  30, 667,  31,   4,
        55, 260,  17,  12, 285, 454,  20,   1, 257,  13,   1, 335,  11,
         1,  67,   3,   1,  13,   1, 490])

In [30]:
df1.loc[1]

Review        Love this dress!  it's sooo pretty.  i happen...
sentiment                                                  pos
Name: 1, dtype: object